In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import face_recognition
import os
import random
import glob
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

from tqdm.auto import tqdm

In [2]:
# Define the input shape
input_shape = (96, 96, 3)

# Define the batch size
batch_size = 32

# Define the number of epochs
epochs = 10

In [3]:
class PAL_Dataset(Dataset):

    def __init__(self, root_dir, hms_dir, transform=None):

        self.root_dir = root_dir
        self.root_dir = hms_dir
        self.transform = transform
        self.classes = sorted([f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))])
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.samples = []

        for cls_idx, cls_name in enumerate(self.classes):
            cls_dir = os.path.join(root_dir, cls_name)
            hm_dir = os.path.join(hms_dir, cls_name)
            for file_name in os.listdir(cls_dir):
                image_path = os.path.join(cls_dir, file_name)
                heatmap_path = os.path.join(hm_dir, str(file_name[:-4])+'.npy')
                self.samples.append((image_path, cls_idx, heatmap_path))


    def __len__(self):
        return len(self.samples)

        

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        file_path, cls_idx, heatmap_path = self.samples[idx]

        
        image = Image.open(file_path)
        heatmap = np.load(heatmap_path)
        
        if self.transform:
            image = self.transform(image)
        

        return image, cls_idx, heatmap

In [4]:
# Define the path to the dataset
data_path = '../datasets/AffectNet'
hm_path = '../datasets/HeatMaps'

# Define the data generator
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor()
    ]),
}

# Define the training data generator
train_dataset = PAL_Dataset(
    root_dir=data_path, 
    hms_dir=hm_path,
    transform=data_transforms['train']
)


train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)


In [5]:
class Cassifier(torch.nn.Module):

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        vgg16 = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        modules = list(vgg16.children())[:-1]
        vgg16 = nn.Sequential(*modules)  # output (512, 7, 7)

        for p in vgg16.parameters():
            p.requires_grad = False

        self.feature_extractor = vgg16

        self.fc1 = nn.Linear(7 * 7 * 512, 4096)
        self.tanh1 = nn.Tanh()
        self.dropout1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(4096, 2048)
        self.tanh2 = nn.Tanh()
        self.dropout2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(2048, 8)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        x1 = self.feature_extractor(x)
        x = x1.view(-1, 7 * 7 * 512)
        x = self.tanh1(self.fc1(x))
        x = self.dropout1(x)
        x = self.tanh2(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        out = self.softmax(x)

        return out, x1

In [6]:
model = Cassifier()
summary(model, (3, 96, 96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 96, 96]           1,792
              ReLU-2           [-1, 64, 96, 96]               0
            Conv2d-3           [-1, 64, 96, 96]          36,928
              ReLU-4           [-1, 64, 96, 96]               0
         MaxPool2d-5           [-1, 64, 48, 48]               0
            Conv2d-6          [-1, 128, 48, 48]          73,856
              ReLU-7          [-1, 128, 48, 48]               0
            Conv2d-8          [-1, 128, 48, 48]         147,584
              ReLU-9          [-1, 128, 48, 48]               0
        MaxPool2d-10          [-1, 128, 24, 24]               0
           Conv2d-11          [-1, 256, 24, 24]         295,168
             ReLU-12          [-1, 256, 24, 24]               0
           Conv2d-13          [-1, 256, 24, 24]         590,080
             ReLU-14          [-1, 256,

In [7]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.05)

In [8]:
# # Train the model with simpl loss
# for epoch in range(epochs):
#     running_loss = 0.0
#     for i, data in tqdm(enumerate(train_loader, 0), total=len(train_loader)):

#         inputs, labels, _ = data

#         optimizer.zero_grad()
#         outputs, _ = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         if i % 10 == 9:
#             tqdm.write(f'loss: {running_loss / 10:.3f}', end='\r')
#             running_loss = 0.0

In [9]:
# # Train the model with simpl loss
# for epoch in range(epochs):
#     running_loss = 0.0
#     for i, data in tqdm(enumerate(train_loader, 0), total=len(train_loader)):

#         inputs, labels, heatmaps = data

#         optimizer.zero_grad()
#         outputs = model(inputs)

#         loss = criterion(outputs, labels)
        
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         if i % 10 == 9:
#             tqdm.write(f'loss: {running_loss / 10:.3f}', end='\r')
#             running_loss = 0.0

In [26]:
inputs, labels, heatmaps = next(iter(train_loader))
optimizer.zero_grad()
inp = inputs[1].requires_grad_()

output, last_conv = model(inp)

C = int(512/2)
mean_third_layer_output = torch.mean(last_conv[C:])
gradient = torch.autograd.grad(mean_third_layer_output, inp, create_graph=True)[0]

mse_loss = nn.MSELoss()
grad_loss = mse_loss(gradient, heatmaps[1])

loss = criterion(output[0], labels[1])


alpha = 1
total_loss = alpha * loss + grad_loss
total_loss = total_loss.float()

total_loss.backward()

optimizer.step()

c:\Users\kobbi\anaconda3\Lib\site-packages\torch\autograd\__init__.py:251: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at ..\torch\csrc\autograd\engine.cpp:1176.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


RuntimeError: Found dtype Double but expected Float

In [15]:
labels[1]

tensor(0)